In [63]:
import time, os, math
from collections import Counter
from itertools import count

class HMM:

    tag_count = {}
    word_tag = []
    word_tag_em = {}
    tags = ["START","B-negative","B-neutral","B-positive","I-negative","I-neutral","I-positive","O","STOP"]
    new_word_tag = ""
    start_y = []
    y_stop = []
    y_sequence = []
    
    def __init__(self,train_x,train_y):
        self.train_x = train_x # train_x is list of words in training data
        self.train_y = train_y # train_y is list of tags in training data related to x
        self.tag_count = Counter(train_y) # total count for each tag found in training data
        self.word_tag = Counter(list(zip(train_x,train_y))) # total count for each word-tag pair found in training data
        self.new_word_tag = self.new_word_emission() #find set probability given to new word

        # find count for each (prev_tag,current_tag) pair found in training data
        self.start_y = train_y[0:len(train_y)-1] 
        self.y_stop = train_y[1:]        
        self.y_sequence = Counter(list(zip(self.start_y,self.y_stop))) 
        #print(self.y_sequence)
        
#=========================== PART 2 =============================
    #counting number of occurence of y_i
    def count_y(self,y_i):                
        return self.tag_count[y_i]
    
    
    #2a count(y pair x)/ count(y)
    def est_emission(self,x_i,y_i):
        return self.word_tag[(x_i,y_i)]
    
    
    #2b emission count(y pair x)/ (count(y)+1)
    def impr_est_emission(self,x_i,y_i):
        #if tag is START or STOP, emission score for empty word = 1
        if y_i == "START" or y_i == "STOP":
            return 1
        elif x_i not in self.train_x:
            return 1/(self.count_y(y_i)+1)
        else:
            return self.word_tag[(x_i,y_i)]/(self.count_y(y_i)+1)
    
    #2b finding tag for new word using estimation 1/ (count(y)+1)   
    def new_word_emission(self):
        em=[]
        for tag in self.tags:  
            #ignore "START" and "STOP"
            if tag == "START" or tag == "STOP":                
                em.append(0)
            else:
                em.append(1/(self.count_y(tag)+1))
        #return label using the index of maximum emission found
        return self.tags[em.index(max(em))]
        
    def train(self):
        #for each (word,tag) pair, calculate emission score and put in dictionary
        for i in self.word_tag:
            self.word_tag_em[i]=self.impr_est_emission(i[0],i[1])
        return self.word_tag_em
            
    #2c
    def sentiment_analysis(self, test_x):
        print("predicting...")
        y_star=[]
        
        #for each word in new training data eg. hello
        for word in test_x:
            #if word is empty, tag = empty (for score calculation)
            if word == "":
                y_star.append("")
                
            #if word is new, add predicted tag for new words
            elif word not in self.train_x:
                y_star.append(self.new_word_tag)
            
            # otherwise, run through all labels excluding START and STOP to find max emission
            else:
                em_list = [0] #account for "START"
                
                #for each tag in tags eg. B+, B-, Bn 
                for tag in self.tags[1:8]:
                    # if (word,tag) pair not available in training data; count(word,tag) = 0 hence emission = 0
                    if (word,tag) not in self.word_tag_em:
                        em_list.append(0)
                    # else look up in emission dictionary calculated in train to find score
                    else:
                        em_list.append(self.word_tag_em[(word,tag)])
                # get max emission and corresponding label        
                index_of_max = em_list.index(max(em_list))
                predicted_tag = self.tags[index_of_max]
                
                # check if label starts with "I". If so, the previous label should start with either "I" or "B"
                # i.e. there should be a previous label and it should not be "O" or ""
                # if not, label should start with "B" instead
                if predicted_tag[0] == "I":
                    if y_star[-1]=="O" or y_star[-1]=="":                           
                        predicted_tag = "B"+predicted_tag[1:]
                    
                        
                    
                y_star.append(predicted_tag)
                                    
        print("done predicting!")
        return y_star

    
#=========================== PART 3 =============================

    def count_transition(self,y_prev,y_current):
        return self.y_sequence[(y_prev,y_current)]
    
    def est_transition(self,y_prev,y_current):
        if y_prev == "STOP" and y_current == "START":
            return 0
        elif (y_prev,y_current) not in self.y_sequence:
            return 0
        else:
            return self.count_transition(y_prev,y_current)/self.count_y(y_prev)

        
    def viterbi(self,tweet):
        n = len(tweet)
        possible_path = {} #(current state k, current tag): (prev tag, score from prev tag)
        score = 0
        optimal_path = []
        # for each time step, calculate maximum score and corresponding preceding tag for each current tag
        for k in range(n+2):
            
            #to prevent underflow use log(score) instead
            #print("k = "+str(k))
            #print("word = "+str(word))
            if k == 0:
                score = math.log(1)
            elif k == 1:
                word = tweet[k-1]
                for tag in self.tags[1:8]: 
                    #print("current tag = "+tag)
                    a = self.est_transition("START",tag)
                    #print("a = "+str(a))
                    if (word,tag) in self.word_tag_em:
                        b = self.word_tag_em[(word,tag)]
                    else:
                        b = self.impr_est_emission(word,tag)
                    #print("b = "+str(b))
                    try:
                        score = math.log(a*b)
                    except ValueError:
                        score = -math.inf
                    possible_path[(1,tag)] = (score,"START")
                    #print(optimal_path[(1,tag)])
            elif k <= n:
                word = tweet[k-1]
                #print("\n==========================================\n")
                for current_tag in self.tags[1:8]:
                    #print("\n----------------------------------------\n")
                    #print("current_tag = "+current_tag)
                    temp = []
                    for prev_tag in self.tags[1:8]:
                        #print("prev_tag = "+prev_tag)
                        a = self.est_transition(prev_tag,current_tag)
                        #print("a = "+str(a))
                        if (word,prev_tag) in self.word_tag_em:
                            b = self.word_tag_em[(word,prev_tag)]
                        else:
                            b = self.impr_est_emission(word,current_tag)
                        #print("b = "+str(b))
                        
                        try: 
                            #print("prev score = "+str(optimal_path[(k-1,prev_tag)][0]))
                            score = possible_path[(k-1,prev_tag)][0]+math.log(a*b)
                            #print("score = "+str(score))
                        except ValueError:
                            score = -math.inf
                            
                        if score != -math.inf:                            
                            temp.append((score,prev_tag))
                    if temp == []:
                        possible_path[(k,current_tag)]=(-math.inf,"O")
                    else:
                        optimal_prev_tag = max(temp)[1]
                        possible_path[(k,current_tag)]=(max(temp)[0],optimal_prev_tag)
            # when k = n+1
            else:
                temp = []
                for tag in self.tags[1:8]: 
                    a = self.est_transition(tag,"STOP")
                    #print("\ntag = "+tag)
                    #print(a)
                    
                    try: 
                        #print(optimal_path[(k-1,tag)])
                        #print("")
                        score = possible_path[(k-1,tag)][0]+math.log(a)
                    except ValueError:
                        score = -math.inf
                        
                    if score != -math.inf:                            
                        temp.append((score,prev_tag))
                if temp == []:
                    possible_path[(k,"STOP")]=(-math.inf,"O")
                else:
                    optimal_prev_tag = max(temp)[1]    
                    possible_path[(k,"STOP")]=(max(temp)[0],optimal_prev_tag)
        current_tag = "STOP"
        for i in range(n+1,1,-1):            
            prev_tag = possible_path[(i,current_tag)][1]
            optimal_path.append(prev_tag)
            current_tag = prev_tag
        #print(possible_path)
        #print(optimal_path)
        optimal_path.reverse()
        #print(optimal_path)
        return optimal_path
        
    def run_viterbi(self,test_data):
        test_tags = []
        for tweet in test_data:
            test_tags.append(self.viterbi(tweet))
        return test_tags
    
    
        




# format: [[words], [tags]]
def read_train(file_name):
    in_file = open(file_name,'r',encoding='utf8')
    l = []
    words = [""]
    tags = ["START"]
    for line in in_file:
        x = line.strip().split()
        if x != []:
            words.append(x[0])
            tags.append(x[1].rstrip('\n'))
        else:
            words.append("")
            tags.append("STOP")
            words.append("")
            tags.append("START")
    tags=tags[0:len(tags)-1]
    words = words[0:len(words)-1]
    l.append(words)
    l.append(tags)
    in_file.close()
    return l
    
#reading and writing to files 
# format:[[tweet1][tweet2]....]
def read_dev_in(file_name):
    in_file = open(file_name,'r',encoding='utf8')
    l = []
    tweet = []
    for line in in_file:
        tweet.append(line.strip())
        if line.strip()=="":
            tweet.remove("")
            l.append(tweet)
            tweet=[]
                
        
    in_file.close()
    return l

def write_devp2(language,word_list,tag_list):
    file_name = language+"/"+"dev.p2.out"
    if os.path.isfile(file_name):
        print('file exist')
        try:
            os.remove(file_name)
            print("deleted file")
        except OSError:
            pass
    out_file = open(file_name,'a',encoding='utf8')

    for i in range(len(word_list)):        
        out_file.write(word_list[i]+" "+tag_list[i]+"\n")
    
    out_file.close()
        
def write_devp3(language,word_list,tag_list):
    file_name = language+"/"+"dev.p3c.out"
    if os.path.isfile(file_name):
        print('file exist')
        try:
            os.remove(file_name)
            print("deleted file")
        except OSError:
            print ("error")
            #pass
    out_file = open(file_name,'a',encoding='utf8')

    for i in range(len(word_list)): 
        for j in range(len(word_list[i])):
            out_file.write(word_list[i][j]+" "+tag_list[i][j]+"\n")
        out_file.write(" \n")
    
    out_file.close()    
train_data = read_train("EN/train")
test_data = read_dev_in("EN/dev.in")

"""
print(train_data[1])
print(len(train_data[1]))
print(train_data[0])
print(len(train_data[0]))
"""
part2_HMM = HMM(train_data[0],train_data[1])
#print(part2_HMM.train())
#print(part2_HMM.sentiment_analysis(test_data))
starttime = time.time()
#part2_HMM.train()
#test_tags = part2_HMM.sentiment_analysis(test_data)
#print(test_data)
#print(part2_HMM.viterbi(test_data))
test_tags = part2_HMM.run_viterbi(test_data)
#write_devp2("EN",test_data,test_tags)
write_devp3("EN",test_data,test_tags)
elapsed = time.time()-starttime

print ("time taken = "+str(elapsed)+"s")



file exist
deleted file
time taken = 58.38954043388367s
